## Using simple Pytorch NeuralNetwork model with a MOA evaluator

* Example showing how a simple Pytorch model can be used with our ```Instance``` representation and MOA evaluator
**Make sure you install Pytorch in your environment (https://pytorch.org/)**

**notebook last updated on 03/12/2023**

In [33]:
from prepare_jpype import start_jpype

start_jpype()

## 0. Reading data and accessing x()

In [34]:
from stream import stream_from_file

DATA_PATH = "./data/"

## Opening a file as a stream
elec_stream = stream_from_file(path_to_csv_or_arff=DATA_PATH+"electricity.csv")

elec_stream.restart()
i = 0
while elec_stream.has_more_instances():
    instance = elec_stream.next_instance()
    if i < 20: # prevent printing all the instances
        print(f'x: {instance.x()}, y: {instance.y()}')
    i+=1

x: [0.       0.056443 0.439155 0.003467 0.422915 0.414912], y: 1.0
x: [0.021277 0.051699 0.415055 0.003467 0.422915 0.414912], y: 1.0
x: [0.042553 0.051489 0.385004 0.003467 0.422915 0.414912], y: 1.0
x: [0.06383  0.045485 0.314639 0.003467 0.422915 0.414912], y: 1.0
x: [0.085106 0.042482 0.251116 0.003467 0.422915 0.414912], y: 0.0
x: [0.106383 0.041161 0.207528 0.003467 0.422915 0.414912], y: 0.0
x: [0.12766  0.041161 0.171824 0.003467 0.422915 0.414912], y: 0.0
x: [0.148936 0.041161 0.152782 0.003467 0.422915 0.414912], y: 0.0
x: [0.170213 0.041161 0.13493  0.003467 0.422915 0.414912], y: 0.0
x: [0.191489 0.041161 0.140583 0.003467 0.422915 0.414912], y: 0.0
x: [0.212766 0.044374 0.168997 0.003467 0.422915 0.414912], y: 1.0
x: [0.234043 0.049868 0.212437 0.003467 0.422915 0.414912], y: 1.0
x: [0.255319 0.051489 0.298721 0.003467 0.422915 0.414912], y: 1.0
x: [0.276596 0.042482 0.39036  0.003467 0.422915 0.414912], y: 0.0
x: [0.297872 0.040861 0.402261 0.003467 0.422915 0.414912], y:

In [35]:
# Getting some extra information about the instance through the MOA representation. 
moa_instance = instance.get_MOA_InstanceExample().getData()
print(f'Number of classes: {moa_instance.numClasses()}')
print(f'Number of features/attributes: {moa_instance.numInputAttributes()}')

for i in range(0, moa_instance.numInputAttributes()):
    print(f'    {moa_instance.attribute(i)}')
    print(f'    {moa_instance.value(i)}')

Number of classes: 2
Number of features/attributes: 6
    @attribute period numeric
    1.0
    @attribute nswprice numeric
    0.050679
    @attribute nswdemand numeric
    0.288753
    @attribute vicprice numeric
    0.003542
    @attribute vicdemand numeric
    0.355256
    @attribute transfer numeric
    0.23114


## 1. Using Pytorch model with MOA evaluator

* Example showing how a simple Pytorch model can be used with our ```Instance``` representation and MOA evaluator
* Uses CPU device
* Model is initialized after receiving the first instance

In [36]:
from evaluation import ClassificationEvaluator
import torch
from torch import nn

# Get cpu device for training.
device = ("cpu")
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size=0, number_of_classes=0):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, number_of_classes)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = None
optimizer = None
loss_fn = nn.CrossEntropyLoss()


# Creating the evaluator
evaluator = ClassificationEvaluator(schema=elec_stream.get_schema())

## Opening a file again to strat from the beginning
elec_stream = stream_from_file(path_to_csv_or_arff=DATA_PATH+"electricity.csv")
i = 0
while elec_stream.has_more_instances():
    i += 1
    instance = elec_stream.next_instance()
    if model is None:
        moa_instance = instance.get_MOA_InstanceExample().getData()
        # initialize the model and send it to the device
        model = NeuralNetwork(input_size=moa_instance.numInputAttributes(), number_of_classes=moa_instance.numClasses()).to(device)
        # set the optimizer
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
        print(model)
    
    X = torch.tensor(instance.x(), dtype=torch.float32)
    y = torch.tensor(instance.y(), dtype=torch.long)
    # set the device and add a dimension to the tensor
    X, y = torch.unsqueeze(X.to(device), 0), torch.unsqueeze(y.to(device),0) 
    
    # turn off gradient collection for test
    with torch.no_grad():
        pred = model(X)
        prediction = torch.argmax(pred)

    # update evaluator with predicted class
    evaluator.update(instance.y(), prediction)
  
    # Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if i % 1000 == 0:
        print(f'Accuracy at {i} : {evaluator.accuracy()}')
    
print(f'Accuracy at {i} : {evaluator.accuracy()}')

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=6, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)
Accuracy at 1000 : 57.3
Accuracy at 2000 : 62.2
Accuracy at 3000 : 62.133333333333326
Accuracy at 4000 : 63.1
Accuracy at 5000 : 63.75999999999999
Accuracy at 6000 : 63.7
Accuracy at 7000 : 63.614285714285714
Accuracy at 8000 : 63.74999999999999
Accuracy at 9000 : 64.33333333333333
Accuracy at 10000 : 64.77000000000001
Accuracy at 11000 : 64.96363636363637
Accuracy at 12000 : 65.28333333333333
Accuracy at 13000 : 65.67692307692307
Accuracy at 14000 : 66.40714285714286
Accuracy at 15000 : 66.90666666666667
Accuracy at 16000 : 67.06875
Accuracy at 17000 : 67.41764705882353
Accuracy at 18000 : 67.9
Accuracy at 19000 : 68.43684210526317
Accuracy at 20000 : 68.975